# Uppgift 3

- Genomsnittligt fall per dag, från början av datsetet till slutet. Använder .sum få totalt antal fall och sedan använder jag .nunique för att få fram antal dagar i datsetet. Antal fall per dag i snitt, löser man med att ta Antal fall / Antal dagar.

In [91]:
import pandas as pd

excel_file_path = "Labb-data/Folkhalsomyndigheten_Covid19.xlsx"


df = pd.read_excel(excel_file_path, sheet_name=0)
total_cases = df["Totalt_antal_fall"].sum()
print(f"Antal fall: {total_cases}")

total_days = df["Statistikdatum"].nunique()
print(f"Antal dagar i datsetet: {total_days}")

average_cases_per_day = int(total_cases / total_days)
print(f"Genomsnittligt antal fall per dag: {average_cases_per_day}")

Antal fall: 2611580
Antal dagar i datsetet: 996
Genomsnittligt antal fall per dag: 2622


### Intensivvård (IVA)
Linjedigramen visar perioder då flest hamnade i IVA eller avled och syftet med datan är för att få en överblick av dödligheten.

Koden läser in blad ett och två från covid19.xlsx. Gör om så datum för avliden och vårdstart blir bara datum för att enklare göra en graf. Kombinerar datum från blad ett och två tillsammans och gör en linjedigram.

In [41]:
import pandas as pd
import plotly.express as px

excel_file_path = "Labb-data/Folkhalsomyndigheten_Covid19.xlsx"

df = pd.read_excel(excel_file_path, sheet_name="Antal avlidna per dag")

df_IVA = pd.read_excel(excel_file_path, sheet_name="Antal intensivvårdade per dag")


df.rename(columns={"Datum_avliden": "Datum"}, inplace=True)

df_IVA.rename(columns={"Datum_vårdstart": "Datum"}, inplace=True)

combined_df = pd.concat([df.set_index("Datum"), df_IVA.set_index("Datum")], axis=1)

combined_df = combined_df.reset_index()

fig = px.area(combined_df, x="Datum", 
              y=["Antal_avlidna", "Antal_intensivvårdade"], 
              labels={"value": "Antal", "variable": "Antal avlidna och i IVA"},
              title="Antal avlidna och patienter i intensivvården ")

fig.show()
fig.write_html("check_mortality_rate.html")

### Antal nya fall och vaccinerade
Syftet med att jämföra fall med vaccination, är för att analysera hur effekttiv covid-19 vaccin är


In [38]:
import pandas as pd
import plotly.express as px

# Load COVID-19 data
covid_data = pd.read_excel("Labb-data/Folkhalsomyndigheten_Covid19.xlsx", sheet_name="Veckodata Riket")

# Load vaccine data
vaccine_data = pd.read_excel("Labb-data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx", sheet_name="Vaccinationer tidsserie")

# Standardize the format of the "År" column in the vaccine data
vaccine_data["År"] = vaccine_data["År"].astype(str).str.replace("2,020", "2020").str.replace("2,021", "2021")

# Filter data for the desired year (e.g., 2021)
desired_year = "2021"
covid_data_2021 = covid_data[covid_data["år"] == int(desired_year)]

# Filter vaccine data for Sweden and compute cumulative sum of "Antal vaccinationer" for each week
vaccine_data_sweden = (
    vaccine_data[(vaccine_data["År"] == desired_year) & (vaccine_data["Region"] == "| Sverige |")]
    .groupby("Vecka")["Antal vaccinationer"]
    .sum()
    .reset_index()
)
vaccine_data_sweden["Cumulative Vaccinations"] = vaccine_data_sweden["Antal vaccinationer"].cumsum()

# Create a copy of the filtered DataFrame to avoid the SettingWithCopyWarning
merged_data_sweden = covid_data_2021.copy()

# Merge with cumulative vaccine data
merged_data_sweden = pd.merge(merged_data_sweden, vaccine_data_sweden, how="inner", left_on="veckonummer", right_on="Vecka")

# Filter out weeks with 0 vaccinations
merged_data_sweden = merged_data_sweden[merged_data_sweden["Antal vaccinationer"] > 0]

# Compute cumulative sum of COVID-19 cases for each week
merged_data_sweden["Cumulative Cases"] = merged_data_sweden["Antal_fall_vecka"].cumsum()

# Filter to include only the first 7 weeks
merged_data_sweden_first_7_weeks = merged_data_sweden.head(7)

# Plotting with Plotly Express for Sweden using a bar plot
fig_sweden = px.bar(merged_data_sweden_first_7_weeks, x="veckonummer", y=["Cumulative Cases", "Cumulative Vaccinations"],
                    title="Cumulative COVID-19 Cases and Vaccinations - First 7 Weeks of 2021 (Sweden)",
                    labels={"value": "Cumulative Antal", "variable": "Kategori", "veckonummer": "Veckonummer"},
                    color_discrete_sequence=["blue", "green"])
                

# Set up a secondary y-axis for "Cumulative Vaccinations"
fig_sweden.update_layout(yaxis2=dict(title="Cumulative Vaccinations", overlaying='y', side='right'))

fig_sweden.show()
